# course  details

In [0]:
import pandas as pd
import boto3
import io
import time
import datetime

#import libraries
import pandas as pd
pd.set_option('display.max_columns', None)
from pandas import Timestamp
import datetime
from datetime import datetime, timezone
import geopandas as gpd
from shapely.geometry import Point,shape,Polygon
from scipy import spatial
import numpy as np
import boto3
import io
import os
import sys
import keplergl
from keplergl import KeplerGl
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
import h3
import networkx as nx

# display the resolution available for h3 indexing with hexagons
def h3_indexTable():
    max_res = 15
    list_hex_edge_km = []
    list_hex_edge_m = []
    list_hex_perimeter_km = []
    list_hex_perimeter_m = []
    list_hex_area_sqkm = []
    list_hex_area_sqm = []

    for i in range(0, max_res + 1):
        ekm = h3.edge_length(resolution=i, unit='km')
        em = h3.edge_length(resolution=i, unit='m')
        list_hex_edge_km.append(round(ekm, 3))
        list_hex_edge_m.append(round(em, 3))
        list_hex_perimeter_km.append(round(6 * ekm, 3))
        list_hex_perimeter_m.append(round(6 * em, 3))

        akm = h3.hex_area(resolution=i, unit='km^2')
        am = h3.hex_area(resolution=i, unit='m^2')
        list_hex_area_sqkm.append(round(akm, 3))
        list_hex_area_sqm.append(round(am, 3))

    df_meta = pd.DataFrame({"edge_length_km": list_hex_edge_km,
                            "perimeter_km": list_hex_perimeter_km,
                            "area_sqkm": list_hex_area_sqkm,
                            "edge_length_m": list_hex_edge_m,
                            "perimeter_m": list_hex_perimeter_m,
                            "area_sqm": list_hex_area_sqm
                            })

    df_meta = df_meta[["edge_length_km", "perimeter_km", "area_sqkm", 
             "edge_length_m", "perimeter_m", "area_sqm"]]
    return(df_meta)

#perform colocation based upon hex bins
def colocate_byH3 (df,res,colo_num):
    #Get Dataframoe to work on and set ts column to utc time
    data_h3 = df.copy(deep=True)
    
    def lat_lng_to_h3(row):
        return h3.geo_to_h3(row['lat'], row['lon'], res)

    def add_geometry(row):
        points = h3.h3_to_geo_boundary(row['hexID'], True)
        return Polygon(points)

    #format ts to datetime
    #data_h3['localized_ts'] = data_h3['localized_ts'].astype(str) # done to plot ts in kepler
    #data_h3['ts'] =  pd.to_datetime(data_h3['ts'].str.strip(), errors='coerce', utc = True)
    data_h3.dropna(subset=['lat','lon','localized_ts'], inplace=True)
    data_h3['localized_ts'] = data_h3['localized_ts'].dt.strftime("%X")

    # Create a new Hex ID column describing what H3 hexagon bin each point falls within
    data_h3['hexID'] = data_h3.apply(lat_lng_to_h3 ,axis=1)

    # Create a column representing the number of unique UDIDs that appear in each hex bin
    data_h3['num_unique'] = (data_h3.groupby('hexID')['uuid'].transform('nunique'))

    # Create a column representing the list of actual unique UDIDs that appear in each hex bin, then create a joined string
    data_h3['Unique_UDID'] = data_h3.groupby('hexID')['uuid'].transform(lambda x: ', '.join(x.unique()))
    data_h3['activity_dates_of_UUIDs'] = data_h3.groupby('hexID')['localized_ts'].transform(lambda x: ', '.join(x.unique()))

    # Create a new narrowed dataset without geometry column - original one was POINT, but we will create a POLYGON one with H3
    hexDF_shape = data_h3[['hexID','uuid','num_unique','activity_dates_of_UUIDs','lat','lon','localized_ts','Unique_UDID']]

    #create a copy to avoid set copy warnings
    hexDF_shape = hexDF_shape.copy()

    # Create POLYGON geometry using H3's h3_to_geo_boundary, then project to WGS 1984 (which is what epsg 4326 represents)
    #
    # apply h3 to geo boundary funciton
    hexDF_shape['geometry'] = hexDF_shape.hexID.apply(lambda x: Polygon(h3.h3_to_geo_boundary(x)))

    #create geodataframe and set crs to wgs 1984
    hexDF_shape = gpd.GeoDataFrame(hexDF_shape, crs=("EPSG:4326"))

    # Filter the dataframe above for only hexagons that have more than one UDID within them
    hexDF_coloc = hexDF_shape[hexDF_shape['num_unique'] > colo_num]
    return(hexDF_coloc)

import time
import datetime

# convert supplied local time to utc time
def Local2UTC(LocalTime):
    # application:  dp6['utcNew'] = dp6['localized_ts'].apply(lambda x: Local2UTC(x))
    EpochSecond = time.mktime(LocalTime.timetuple())
    utcTime = datetime.datetime.utcfromtimestamp(EpochSecond)
    return utcTime

def clean(crse,name):
    #csv file clean-up
    filePath = "course_%s_%s_allData.csv" % (crse, name)
    #remove the file if exists
    if os.path.exists(filePath):
        os.remove(filePath)
    filePath = "course_%s_allStudent_data.csv" % crse
    #remove the file if exists
    if os.path.exists(filePath):
        os.remove(filePath)


#for use in offline mode
offline_lookup = pd.DataFrame({"name":['Aelfric','Alfred','Brida','Guthrum','Jkricket','Odda','Ragnar','Ubba'
                                              ,'Uhtred']
                               ,"udid":['801b9140-6179-4eae-b6db-fcbf5788ce0c'
                                       ,'f2957e59-a638-4e6a-92cc-9a6802c4888f'
                                       ,'bea8836d-3008-44a3-8c3c-53555f0a2dee'
                                       ,'22c07d36-5d29-4064-a3d7-f9019cd27cea'
                                       ,'ce3bd3de-8031-4b59-93f8-445a2601780c'
                                       ,'66946627-b308-4b54-a147-5f4983197b73'
                                       ,'32bdb1af-fd87-4096-a683-63cd16a264d6'
                                       ,'a266ab79-daf2-4375-a94e-5522a980e934'
                                       ,'e30234a8-a2f8-4bdd-b82e-3ac919eeec7a']})


#----------------------------------------------------------------------------------
#          Lookup for the mid by the students nobebook' name
#----------------------------------------------------------------------------------
def get_mid(dev_name,crse):
    student_uuid_list = "Student_UDID_List.csv"
    #get device_name uuid
    obj = s3.get_object(Bucket=awsBucket, Key=student_uuid_list)
    student_uuids = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0, dtype = {
        'UDID':str
        ,'Name':str
    })

    #rename columns
    student_uuids.rename(columns ={
        "UDID":"uuid"
        ,"Name":"name"
    }, inplace = True)

    #lower columns and remove any white space
    student_uuids['uuid'] = student_uuids['uuid'].str.strip().str.lower()
    #lookup the uuid by the name of the students device name
    lookup = student_uuids['name'] == '%s' % dev_name 
    #filter data
    student_uuids.where(lookup, inplace=True)
    #drop nan
    student_uuids = student_uuids.dropna()
    #get the mid for the student based upon the name of device
    maid = student_uuids.iloc[0]['uuid']
    return(maid)

#----------------------------------------------------------------------------------
#          Read all student IOT device data
#----------------------------------------------------------------------------------
def get_IoT_data(crse,start,end):
    #deviceName_Leofric
    try:
        deviceName_Leofric = "%s/leofric_iot.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Leofric)
        student_iot_Leofric = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         , dtype ={'MAC':str,'FirstSeen':str,'SSID':str,'AuthMode':str,'Channel':float,'RSSI':float
                                   ,'CurrentLatitude':float,'CurrentLongitude':float,'AltitudeMeters':float,'Type':str
                                  }, low_memory=False)
        student_iot_Leofric['name'] = 'Leofric'
    except: 
        student_iot_Leofric = pd.DataFrame(columns = ['name','MAC','FirstSeen','SSID','AuthMode','Channel','RSSI','CurrentLatitude'
                                                      ,'AltitudeMeters','Type'])
    #deviceName_Guthrum
    try:
        deviceName_Guthrum = "%s/guthrum_iot.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Guthrum)
        student_iot_Guthrum = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         , dtype ={'MAC':str,'FirstSeen':str,'SSID':str,'AuthMode':str,'Channel':float,'RSSI':float
                                   ,'CurrentLatitude':float,'CurrentLongitude':float,'AltitudeMeters':float,'Type':str
                                  }, low_memory=False)
        student_iot_Guthrum['name'] = 'Guthrum'
    except: 
        student_iot_Guthrum = pd.DataFrame(columns = ['name','MAC','FirstSeen','SSID','AuthMode','Channel','RSSI','CurrentLatitude'
                                                      ,'AltitudeMeters','Type'])
    #deviceName_Jkricket  
    try:
        deviceName_Jkricket = "%s/_jkricket_iot.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Jkricket)
        student_iot_Jkricket = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         , dtype ={'MAC':str,'FirstSeen':str,'SSID':str,'AuthMode':str,'Channel':float,'RSSI':float
                                   ,'CurrentLatitude':float,'CurrentLongitude':float,'AltitudeMeters':float,'Type':str
                                  }, low_memory=False)
        student_iot_Jkricket['name'] = 'Jkricket'
    except: 
        student_iot_Jkricket = pd.DataFrame(columns = ['name','MAC','FirstSeen','SSID','AuthMode','Channel','RSSI','CurrentLatitude'
                                                      ,'AltitudeMeters','Type'])
    #deviceName_Alfred
    try:
        deviceName_Alfred = "%s/alfred_iot.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Alfred)
        student_iot_Alfred = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         , dtype ={'MAC':str,'FirstSeen':str,'SSID':str,'AuthMode':str,'Channel':float,'RSSI':float
                                   ,'CurrentLatitude':float,'CurrentLongitude':float,'AltitudeMeters':float,'Type':str
                                  }, low_memory=False)
        student_iot_Alfred['name'] = 'Alfred'
    except: 
        student_iot_Alfred = pd.DataFrame(columns = ['name','MAC','FirstSeen','SSID','AuthMode','Channel','RSSI','CurrentLatitude'
                                                      ,'AltitudeMeters','Type'])

    #deviceName_Ragnar    
    try:
        deviceName_Ragnar = "%s/ragnar_iot.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Ragnar)
        student_iot_Ragnar = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         , dtype ={'MAC':str,'FirstSeen':str,'SSID':str,'AuthMode':str,'Channel':float,'RSSI':float
                                   ,'CurrentLatitude':float,'CurrentLongitude':float,'AltitudeMeters':float,'Type':str
                                  }, low_memory=False)
        student_iot_Ragnar['name'] = 'Ragnar'
    except: 
        student_iot_Ragnar = pd.DataFrame(columns = ['name','MAC','FirstSeen','SSID','AuthMode','Channel','RSSI','CurrentLatitude'
                                                      ,'AltitudeMeters','Type'])    
    #deviceName_Brida
    try:
        deviceName_Brida = "%s/brida_iot.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Brida)
        student_iot_Brida = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         , dtype ={'MAC':str,'FirstSeen':str,'SSID':str,'AuthMode':str,'Channel':float,'RSSI':float
                                   ,'CurrentLatitude':float,'CurrentLongitude':float,'AltitudeMeters':float,'Type':str
                                  }, low_memory=False)
        student_iot_Brida['name'] = 'Brida'
    except: 
        student_iot_Brida = pd.DataFrame(columns = ['name','MAC','FirstSeen','SSID','AuthMode','Channel','RSSI','CurrentLatitude'
                                                      ,'AltitudeMeters','Type'])   

    #deviceName_Odda
    try:
        deviceName_Odda = "%s/odda_iot.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Odda)
        student_iot_Odda = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         , dtype ={'MAC':str,'FirstSeen':str,'SSID':str,'AuthMode':str,'Channel':float,'RSSI':float
                                   ,'CurrentLatitude':float,'CurrentLongitude':float,'AltitudeMeters':float,'Type':str
                                  }, low_memory=False)
        student_iot_Odda['name'] = 'Odda'
    except: 
        student_iot_Odda = pd.DataFrame(columns = ['name','MAC','FirstSeen','SSID','AuthMode','Channel','RSSI','CurrentLatitude'
                                                      ,'AltitudeMeters','Type'])

    #deviceName_Athelflaed
    try:
        deviceName_Athelflaed = "%s/athelflaed_iot.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Athelflaed)
        student_iot_Athelflaed = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         , dtype ={'MAC':str,'FirstSeen':str,'SSID':str,'AuthMode':str,'Channel':float,'RSSI':float
                                   ,'CurrentLatitude':float,'CurrentLongitude':float,'AltitudeMeters':float,'Type':str
                                  }, low_memory=False)
        student_iot_Athelflaed['name'] = 'Athelflaed'
    except: 
        student_iot_Athelflaed = pd.DataFrame(columns = ['name','MAC','FirstSeen','SSID','AuthMode','Channel','RSSI','CurrentLatitude'
                                                      ,'AltitudeMeters','Type']) 
    #deviceName_Hild
    try:
        deviceName_Hild = "%s/hild_iot.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Hild)
        student_iot_Hild = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         , dtype ={'MAC':str,'FirstSeen':str,'SSID':str,'AuthMode':str,'Channel':float,'RSSI':float
                                   ,'CurrentLatitude':float,'CurrentLongitude':float,'AltitudeMeters':float,'Type':str
                                  }, low_memory=False)
        student_iot_Hild['name'] = 'Hild'
    except: 
        student_iot_Hild = pd.DataFrame(columns = ['name','MAC','FirstSeen','SSID','AuthMode','Channel','RSSI','CurrentLatitude'
                                                      ,'AltitudeMeters','Type']) 
    #deviceName_Ubba
    try:
        deviceName_Ubba = "%s/ubba_iot.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Ubba)
        student_iot_Ubba = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         , dtype ={'MAC':str,'FirstSeen':str,'SSID':str,'AuthMode':str,'Channel':float,'RSSI':float
                                   ,'CurrentLatitude':float,'CurrentLongitude':float,'AltitudeMeters':float,'Type':str
                                  }, low_memory=False)
        student_iot_Ubba['name'] = 'Ubba'
    except: 
        student_iot_Ubba = pd.DataFrame(columns = ['name','MAC','FirstSeen','SSID','AuthMode','Channel','RSSI','CurrentLatitude'
                                                      ,'AltitudeMeters','Type'])  
    #deviceName_Aelfric
    try:
        deviceName_Aelfric = "%s/aelfric_iot.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Aelfric)
        student_iot_Aelfric = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         , dtype ={'MAC':str,'FirstSeen':str,'SSID':str,'AuthMode':str,'Channel':float,'RSSI':float
                                   ,'CurrentLatitude':float,'CurrentLongitude':float,'AltitudeMeters':float,'Type':str
                                  }, low_memory=False)
        student_iot_Aelfric['name'] = 'Aelfric'
    except: 
        student_iot_Aelfric = pd.DataFrame(columns = ['name','MAC','FirstSeen','SSID','AuthMode','Channel','RSSI','CurrentLatitude'
                                                      ,'AltitudeMeters','Type'])
    #deviceName_Aelfric
    try:
        deviceName_Uhtred = "%s/uhtred_iot.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Uhtred)
        student_iot_Uhtred = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         , dtype ={'MAC':str,'FirstSeen':str,'SSID':str,'AuthMode':str,'Channel':float,'RSSI':float
                                   ,'CurrentLatitude':float,'CurrentLongitude':float,'AltitudeMeters':float,'Type':str
                                  }, low_memory=False)
        student_iot_Uhtred['name'] = 'Uhtred'
    except: 
        student_iot_Uhtred = pd.DataFrame(columns = ['name','MAC','FirstSeen','SSID','AuthMode','Channel','RSSI','CurrentLatitude'
                                                      ,'AltitudeMeters','Type'])



    # Concat dataframes into one for combined data set from source
    student_iot = pd.concat([student_iot_Leofric, student_iot_Guthrum, student_iot_Jkricket,student_iot_Alfred,student_iot_Ragnar
                   ,student_iot_Brida,student_iot_Odda,student_iot_Athelflaed,student_iot_Hild,student_iot_Ubba
                   ,student_iot_Aelfric,student_iot_Uhtred])

    #reset the index due to concating multiple dataframes together
    student_iot = student_iot.reset_index()

    #rename columns to lowercase standard
    student_iot.rename(columns = {
        "MAC" : "mac"
        ,"SSID":"ssid"
        ,"AuthMode":"authMode"
        ,"FirstSeen":"localized_ts"
        ,"Channel":"channel"
        ,"RSSI":"rssi"
        ,"CurrentLatitude":"lat"
        ,"CurrentLongitude":"lon"
        ,"AltitudeMeters":"altitude"
        ,"AccuracyMeters":"accuracy"
        ,"Type":"type"
    }, inplace = True)

    #Add Source Column
    student_iot['source'] = 'iot'

    # lower case columns and strip out any whitespace that may be present
    student_iot['mac'] = student_iot['mac'].str.strip().str.lower()
    student_iot['ssid'] = student_iot['ssid'].str.strip().str.lower()
    student_iot['authMode'] = student_iot['authMode'].str.strip().str.lower()
    student_iot['type'] = student_iot['type'].str.strip().str.lower()

    #covert to_datetime
    student_iot['tzID'] = 'US/Eastern'
    # --------- original
    #student_iot['localized_ts'] =  pd.to_datetime(student_iot['localized_ts'].str.strip(), errors='coerce', utc = True)
    #
    #localize the timezone id
    #student_iot['localized_ts'] = student_iot['localized_ts'].dt.tz_convert('US/Eastern')
    #
    #---- New code
    #
    student_iot['localized_ts'] = pd.to_datetime(student_iot['localized_ts'].str.strip(), errors ='coerce',utc = True)
    #enforce timezone as us/eastern
    student_iot['localized_ts'] = student_iot['localized_ts'].dt.tz_convert('US/Eastern')+pd.Timedelta(hours=4)
    # --- end new code
    student_iot['dtg'] = pd.to_datetime(student_iot['localized_ts'])
    student_iot['dtg'] = student_iot['dtg'].dt.date
    student_iot['dtg'] = pd.to_datetime(student_iot['dtg'])
    #make sure we don't have any data violating epoch time
    mask = (student_iot['dtg'] >= start) & (student_iot['dtg'] <=end)
    student_iot = student_iot.loc[mask]
    #convert local time to utc time
    student_iot['utc_ts'] = student_iot['localized_ts'].apply(lambda x: Local2UTC(x))
    #add in unix_timestamp
    student_iot['unix_timestamp'] = student_iot['utc_ts'].apply(lambda x: x.timestamp())
    student_iot['unix_timestamp'] = student_iot['unix_timestamp'].astype(int)

    # ----------- read name to device master file --------------------
    student_uuid_list = "Student_UDID_List.csv"
    #get device_name uuid
    obj = s3.get_object(Bucket=awsBucket, Key=student_uuid_list)
    student_uuids = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0, dtype = {
        'UDID':str
        ,'Name':str
    })

    #rename columns
    student_uuids.rename(columns ={
        "UDID":"uuid"
        ,"Name":"name"
    }, inplace = True)

    #lower columns and remove any white space
    student_uuids['uuid'] = student_uuids['uuid'].str.strip().str.lower()

    # ----------------------- Join the two dataframes together -----------------------
    student_iot = pd.merge(student_iot,student_uuids,on='name')

    #reorder columns
    student_iot = student_iot[['dtg','name','uuid','source','lat','lon','unix_timestamp','utc_ts','localized_ts','tzID','type','ssid'
                               ,'mac','altitude','accuracy','rssi','channel','authMode']]
    
    # deal with nan
    student_iot = student_iot.fillna('')
    
    #return the results
    return(student_iot)

#----------------------------------------------------------------------------------
#          Read all student App device data
#----------------------------------------------------------------------------------
def get_app_data(crse,start,end):
    #deviceName_Leofric
    try:
        deviceName_Leofric = "%s/Leofric_App.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Leofric)
        student_app_Leofric = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         ,dtype ={'Valid':str,'Time':str,'Latitude':float,'Longitude':float,'Altitude':str
                                  ,'Accuracy':str,'Speed':str,'Address':str}, low_memory=False)
        student_app_Leofric['name'] = 'Leofric'
    except: 
        student_app_Leofric = pd.DataFrame(columns = ['name','Valid','Time','Latitude','Longitude','Altitude','Accuracy'
                                                      ,'Speed','Address'])
    #deviceName_Guthrum
    try:
        deviceName_Guthrum = "%s/Guthrum_App.csv" % crse
        obj = s3.get_object(Bucket=bucket, Key=deviceName_Guthrum)
        student_app_Guthrum = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         ,dtype ={'Valid':str,'Time':str,'Latitude':float,'Longitude':float,'Altitude':str
                                  ,'Accuracy':str,'Speed':str,'Address':str}, low_memory=False)
        student_app_Guthrum['name'] = 'Guthrum'
    except: 
        student_app_Guthrum = pd.DataFrame(columns = ['name','Valid','Time','Latitude','Longitude','Altitude','Accuracy'
                                                      ,'Speed','Address'])
    #deviceName_Jkricket  
    try:
        deviceName_Jkricket = "%s/_Jkricket_App.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Jkricket)
        student_app_Jkricket = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         ,dtype ={'Valid':str,'Time':str,'Latitude':float,'Longitude':float,'Altitude':str
                                  ,'Accuracy':str,'Speed':str,'Address':str}, low_memory=False)
        student_app_Jkricket['name'] = 'Jkricket'
    except: 
        student_app_Jkricket = pd.DataFrame(columns = ['name','Valid','Time','Latitude','Longitude','Altitude','Accuracy'
                                                      ,'Speed','Address'])        
        
    #deviceName_Alfred
    try:
        deviceName_Alfred = "%s/Alfred_App.csv" % (crse)       
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Alfred)
        student_app_Alfred = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         ,dtype ={'Valid':str,'Time':str,'Latitude':float,'Longitude':float,'Altitude':str
                                  ,'Accuracy':str,'Speed':str,'Address':str}, low_memory=False)
        student_app_Alfred['name'] = 'Alfred'
    except: 
        student_app_Alfred = pd.DataFrame(columns = ['name','Valid','Time','Latitude','Longitude','Altitude','Accuracy'
                                                      ,'Speed','Address'])               
    #deviceName_Ragnar    
    try:
        deviceName_Ragnar = "%s/Ragnar_App.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Ragnar)       
        student_app_Ragnar = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         ,dtype ={'Valid':str,'Time':str,'Latitude':float,'Longitude':float,'Altitude':str
                                  ,'Accuracy':str,'Speed':str,'Address':str}, low_memory=False)
        student_app_Ragnar['name'] = 'Ragnar'
    except: 
        student_app_Ragnar = pd.DataFrame(columns = ['name','Valid','Time','Latitude','Longitude','Altitude','Accuracy'
                                                   ,'Speed','Address'])      
   #deviceName_Brida
    try:
        deviceName_Brida = "%s/Brida_App.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Brida) 
        student_app_Brida = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         ,dtype ={'Valid':str,'Time':str,'Latitude':float,'Longitude':float,'Altitude':str
                                  ,'Accuracy':str,'Speed':str,'Address':str}, low_memory=False)
        student_app_Brida['name'] = 'Brida'
    except: 
        student_app_Brida = pd.DataFrame(columns = ['name','Valid','Time','Latitude','Longitude','Altitude','Accuracy'
                                                   ,'Speed','Address'])   
        
    #deviceName_Odda
    try:
        deviceName_Odda = "%s/Odda_App.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Odda) 
        student_app_Odda = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         ,dtype ={'Valid':str,'Time':str,'Latitude':float,'Longitude':float,'Altitude':str
                                  ,'Accuracy':str,'Speed':str,'Address':str}, low_memory=False)
        student_app_Odda['name'] = 'Odda'
    except: 
        student_app_Odda = pd.DataFrame(columns = ['name','Valid','Time','Latitude','Longitude','Altitude','Accuracy'
                                                   ,'Speed','Address']) 
        
    #deviceName_Athelflaed
    try:
        deviceName_Athelflaed = "%s/Athelflaed_App.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Athelflaed)       
        student_app_Athelflaed = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         ,dtype ={'Valid':str,'Time':str,'Latitude':float,'Longitude':float,'Altitude':str
                                  ,'Accuracy':str,'Speed':str,'Address':str}, low_memory=False)
        student_app_Athelflaed['name'] = 'Athelflaed'
    except: 
        student_app_Athelflaed = pd.DataFrame(columns = ['name','Valid','Time','Latitude','Longitude','Altitude','Accuracy'
                                                   ,'Speed','Address'])        
    #deviceName_Hild
    try:
        deviceName_Hild = "%s/Hild_App.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Hild)        
        student_app_Hild = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                         ,dtype ={'Valid':str,'Time':str,'Latitude':float,'Longitude':float,'Altitude':str
                                  ,'Accuracy':str,'Speed':str,'Address':str}, low_memory=False)
        student_app_Hild['name'] = 'Hild'
    except: 
        student_app_Hild = pd.DataFrame(columns = ['name','Valid','Time','Latitude','Longitude','Altitude','Accuracy'
                                                   ,'Speed','Address'])             
    #deviceName_Ubba
    try:
        deviceName_Ubba = "%s/Ubba_App.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Ubba)
        student_app_Ubba = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                                         ,dtype ={'Valid':str,'Time':str,'Latitude':float,'Longitude':float
                                                  ,'Altitude':str,'Accuracy':str,'Speed':str,'Address':str}, low_memory=False)
        student_app_Ubba['name'] = 'Ubba'
    except: 
        student_app_Ubba = pd.DataFrame(columns = ['name','Valid','Time','Latitude','Longitude','Altitude','Accuracy'
                                                   ,'Speed','Address']) 
    #deviceName_Aelfric
    try:
        deviceName_Aelfric = "%s/Aelfric_App.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Aelfric)        
        student_app_Aelfric = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                                         ,dtype ={'Valid':str,'Time':str,'Latitude':float,'Longitude':float
                                                  ,'Altitude':str,'Accuracy':str,'Speed':str,'Address':str}, low_memory=False)
        student_app_Aelfric['name'] = 'Aelfric'
    except: 
        student_app_Aelfric = pd.DataFrame(columns = ['name','Valid','Time','Latitude','Longitude','Altitude','Accuracy'
                                                   ,'Speed','Address'])
       #deviceName_Uhtred
    try:
        deviceName_Uhtred = "%s/Uhtred_App.csv" % (crse)
        obj = s3.get_object(Bucket=awsBucket, Key=deviceName_Uhtred)        
        student_app_Uhtred = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0
                                         ,dtype ={'Valid':str,'Time':str,'Latitude':float,'Longitude':float
                                                  ,'Altitude':str,'Accuracy':str,'Speed':str,'Address':str}, low_memory=False)
        student_app_Uhtred['name'] = 'Uhtred'
    except: 
        student_app_Uhtred = pd.DataFrame(columns = ['name','Valid','Time','Latitude','Longitude','Altitude','Accuracy'
                                                   ,'Speed','Address'])
        
    # Concat dataframes into one for combined data set from source
    student_app = pd.concat([student_app_Leofric, student_app_Guthrum,student_app_Jkricket,student_app_Ragnar
                            ,student_app_Brida,student_app_Odda,student_app_Athelflaed,student_app_Hild
                            ,student_app_Ubba,student_app_Aelfric,student_app_Uhtred])
 
    #Add Source Column
    student_app['source'] = 'app'   
    #format the dataframe
    student_app['Altitude'] = student_app['Altitude'].replace(' m','', regex=True).astype(float)
    student_app['Accuracy'] = student_app['Accuracy'].replace(' m','', regex=True).astype(float)
    student_app['Speed'] = student_app['Speed'].replace(' mph','', regex=True).astype(float)
    student_app['Address'] = student_app['Address'].replace('°','', regex=True).astype(str)
 
    #rename columns to lowercase standard
    student_app.rename(columns = {
        "Valid":"valid"
        ,"Time":"localized_ts"
        ,"Latitude":"lat"
        ,"Longitude":"lon"
        ,"Altitude":"altitude"
        ,"Accuracy":"accuracy"
        ,"Speed":"speed"
        ,"Address":"latLon_string"
    }, inplace = True)
    # lower case columns and strip out any whitespace that may be present
    student_app['valid'] = student_app['valid'].str.strip().str.lower()
    student_app['latLon_string'] = student_app['latLon_string'].str.strip().str.lower()

    #covert to_datetime
    student_app['tzID'] = 'US/Eastern'
    #
    # -------- original code
    #student_app['localized_ts'] =  pd.to_datetime(student_app['localized_ts'].str.strip(), errors='coerce', utc = True)
    #localize the timezone id
    #student_app['localized_ts'] = student_app['localized_ts'].dt.tz_convert('US/Eastern')
    #
    # ---------- end original code
    #
    # ----------- new code
    # convert to timezone aware dataframe
    student_app['localized_ts'] = pd.to_datetime(student_app['localized_ts'].str.strip(), errors ='coerce',utc = True)
    #enforce timezone as us/eastern
    student_app['localized_ts'] = student_app['localized_ts'].dt.tz_convert('US/Eastern')+pd.Timedelta(hours=5)
    #
    # ----- end new code
    student_app['dtg'] = pd.to_datetime(student_app['localized_ts'])
    student_app['dtg'] = student_app['dtg'].dt.date
    student_app['dtg'] = pd.to_datetime(student_app['dtg'])
    #make sure we don't have any data violating epoch time
    mask = (student_app['dtg'] >= start) & (student_app['dtg'] <=end)
    student_app = student_app.loc[mask]
    #convert local time to utc time
    student_app['utc_ts'] = student_app['localized_ts'].apply(lambda x: Local2UTC(x))
    #add in unix_timestamp
    student_app['unix_timestamp'] = student_app['utc_ts'].apply(lambda x: x.timestamp())
    student_app['unix_timestamp'] = student_app['unix_timestamp'].astype(int)
    #
    # ----------- read name to device master file --------------------
    student_uuid_list = "Student_UDID_List.csv"
    #get device_name uuid
    obj = s3.get_object(Bucket=awsBucket, Key=student_uuid_list)
    student_uuids = pd.read_csv(io.BytesIO(obj['Body'].read()),header=0, dtype = {
        'UDID':str
        ,'Name':str
    })

    #rename columns
    student_uuids.rename(columns ={
        "UDID":"uuid"
        ,"Name":"name"
    }, inplace = True)

    #lower columns and remove any white space
    student_uuids['uuid'] = student_uuids['uuid'].str.strip().str.lower()

    # ----------------------- Join the two dataframes together -----------------------
    student_app = pd.merge(student_app,student_uuids,on='name')
    #reorder columns
    student_app = student_app[['dtg','name','uuid','source','lat','lon','unix_timestamp','utc_ts','localized_ts','tzID','altitude'
                               ,'accuracy','speed','latLon_string','valid']]
    
    # deal with nan
    student_app = student_app.fillna('')
    #return the dataframe as results of union of all app student data with uuid join by name
    return(student_app)

#Merge the data together
def mergeAll_data(student_appAll,student_iotAll):
    dataAll = pd.concat([student_appAll,student_iotAll])
    #reset index from concat function
    dataAll.reset_index(inplace = True, drop = True)

    # deal with the nan
    dataAll[['lat','lon','accuracy','gps_speed','altitude','speed']] = dataAll[['lat','lon','accuracy','gps_speed'
                                                                                ,'altitude','speed']].fillna(value=0)
    dataAll = dataAll.fillna('')

    #format date columns
    dataAll['utc_ts'] = dataAll['utc_ts'].astype(str)
    dataAll['utc_ts'] =  pd.to_datetime(dataAll['utc_ts'].str.strip(), errors='coerce', utc = True)

    #formate datatypes
    dataAll['lat'] = dataAll['lat'].astype(float)
    dataAll['lon'] = dataAll['lon'].astype(float)
    dataAll['altitude'] = dataAll['altitude'].astype(float)
    dataAll['speed'] = dataAll['speed'].astype(float)

    #add a unique ID to dataframe
    dataAll.insert(0, 'id', range(0, 0 + len(dataAll)))
    dataAll = dataAll.fillna('')
    
    #return dataframe
    return(dataAll)

# kepler config
config1 = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "euwj4t8",
          "type": "point",
          "config": {
            "dataId": "all_data",
            "label": "Point",
            "color": [
              18,
              147,
              154
            ],
            "columns": {
              "lat": "lat",
              "lng": "lon",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "ColorBrewer Paired-12",
                "type": "qualitative",
                "category": "ColorBrewer",
                "colors": [
                  "#a6cee3",
                  "#1f78b4",
                  "#b2df8a",
                  "#33a02c",
                  "#fb9a99",
                  "#e31a1c",
                  "#fdbf6f",
                  "#ff7f00",
                  "#cab2d6",
                  "#6a3d9a",
                  "#ffff99",
                  "#b15928"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                0,
                50
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "name",
              "type": "string"
            },
            "colorScale": "ordinal",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "all_data": [
              {
                "name": "name",
                "format": None
              },
              {
                "name": "uuid",
                "format": None
              },
              {
                "name": "localized_ts",
                "format": None
              },
              {
                "name": "lat",
                "format": None
              },
              {
                "name": "lon",
                "format": None
              },
              {
                "name": "source",
                "format": None
              },
              {
                "name": "app",
                "format": None
              },
              {
                "name": "user_agent",
                "format": None
              },
              {
                "name": "ipv_4",
                "format": None
              },
              {
                "name": "carrier",
                "format": None
              },
              {
                "name": "keywords",
                "format": None
              },
              {
                "name": "altitude",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}
config2 = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "mdhffdp",
          "type": "point",
          "config": {
            "dataId": "time_filtered_data",
            "label": "Point",
            "color": [
              221,
              178,
              124
            ],
            "columns": {
              "lat": "lat",
              "lng": "lon",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Ice And Fire",
                "type": "diverging",
                "category": "Uber",
                "colors": [
                  "#0198BD",
                  "#49E3CE",
                  "#E8FEB5",
                  "#FEEDB1",
                  "#FEAD54",
                  "#D50255"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                0,
                50
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "source",
              "type": "string"
            },
            "colorScale": "ordinal",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "time_filtered_data": [
              {
                "name": "id",
                "format": None
              },
              {
                "name": "name",
                "format": None
              },
              {
                "name": "uuid",
                "format": None
              },
              {
                "name": "source",
                "format": None
              },
              {
                "name": "localized_ts",
                "format": None
              },
              {
                "name": "ipv_4",
                "format": None
              },
              {
                "name": "carrier",
                "format": None
              },
              {
                "name": "user_agent",
                "format": None
              },
              {
                "name": "ssid",
                "format": None
              },
              {
                "name": "keywords",
                "format": None
              },
              {
                "name": "app",
                "format": None
              },
              {
                "name": "place_name",
                "format": None
              },
              {
                "name": "lat",
                "format": None
              },
              {
                "name": "lon",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}
config3 = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "0ccvbn",
          "type": "point",
          "config": {
            "dataId": "two_devices",
            "label": "Point",
            "color": [
              130,
              154,
              227
            ],
            "columns": {
              "lat": "lat",
              "lng": "lon",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Ice And Fire",
                "type": "diverging",
                "category": "Uber",
                "colors": [
                  "#0198BD",
                  "#49E3CE",
                  "#E8FEB5",
                  "#FEEDB1",
                  "#FEAD54",
                  "#D50255"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                0,
                50
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "name",
              "type": "string"
            },
            "colorScale": "ordinal",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "two_devices": [
              {
                "name": "dtg",
                "format": None
              },
              {
                "name": "name",
                "format": None
              },
              {
                "name": "uuid",
                "format": None
              },
              {
                "name": "source",
                "format": None
              },
              {
                "name": "lat",
                "format": None
              },
              {
                "name": "lon",
                "format": None
              },
              {
                "name": "localized_ts",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}
config_yourData = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "o12ushs",
          "type": "point",
          "config": {
            "dataId": "rtb_data",
            "label": "rtb_data",
            "color": [
              227,
              26,
              26
            ],
            "columns": {
              "lat": "lat",
              "lng": "lon",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                0,
                50
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": []
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear"
          }
        },
        {
          "id": "83sy8io",
          "type": "point",
          "config": {
            "dataId": "sdk_data",
            "label": "sdk_data",
            "color": [
              214,
              82,
              0
            ],
            "columns": {
              "lat": "lat",
              "lng": "lon",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                0,
                50
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear"
          }
        },
        {
          "id": "jyp056",
          "type": "point",
          "config": {
            "dataId": "yourdp6SDK_source",
            "label": "dp6_sdk",
            "color": [
              207,
              23,
              80
            ],
            "columns": {
              "lat": "lat",
              "lng": "lon",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                0,
                50
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear"
          }
        },
        {
          "id": "yl7d618p",
          "type": "point",
          "config": {
            "dataId": "app_data",
            "label": "app_data",
            "color": [
              23,
              184,
              190
            ],
            "columns": {
              "lat": "lat",
              "lng": "lon",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                0,
                50
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear"
          }
        },
        {
          "id": "7oclpdu",
          "type": "point",
          "config": {
            "dataId": "IOT_data",
            "label": "iot_data",
            "color": [
              254,
              242,
              26
            ],
            "columns": {
              "lat": "lat",
              "lng": "lon",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                0,
                50
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "rtb_data": [
              {
                "name": "name",
                "format": None
              },
              {
                "name": "uuid",
                "format": None
              },
              {
                "name": "source",
                "format": None
              },
              {
                "name": "lat",
                "format": None
              },
              {
                "name": "lon",
                "format": None
              },
              {
                "name": "localized_ts",
                "format": None
              }
            ],
            "sdk_data": [
              {
                "name": "name",
                "format": None
              },
              {
                "name": "uuid",
                "format": None
              },
              {
                "name": "source",
                "format": None
              },
              {
                "name": "lat",
                "format": None
              },
              {
                "name": "lon",
                "format": None
              },
              {
                "name": "localized_ts",
                "format": None
              }
            ],
            "yourdp6SDK_source": [
              {
                "name": "name",
                "format": None
              },
              {
                "name": "uuid",
                "format": None
              },
              {
                "name": "source",
                "format": None
              },
              {
                "name": "lat",
                "format": None
              },
              {
                "name": "lon",
                "format": None
              },
              {
                "name": "localized_ts",
                "format": None
              }
            ],
            "app_data": [
              {
                "name": "name",
                "format": None
              },
              {
                "name": "uuid",
                "format": None
              },
              {
                "name": "source",
                "format": None
              },
              {
                "name": "lat",
                "format": None
              },
              {
                "name": "lon",
                "format": None
              }
            ],
            "IOT_data": [
              {
                "name": "name",
                "format": None
              },
              {
                "name": "uuid",
                "format": None
              },
              {
                "name": "source",
                "format": None
              },
              {
                "name": "lat",
                "format": None
              },
              {
                "name": "lon",
                "format": None
              },
              {
                "name": "localized_ts",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 38.89272523796748,
      "longitude": -77.22862603954887,
      "pitch": 0,
      "zoom": 12.021321720342172,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

print('Module Import Completed')

Module Import Completed

In [0]:

course = 4

#online (1) or offline (0)
connection = 0

# for online connections you have the option to download all course data to the computer.
# download(1) skip_download(0)
download_csv = 0

# Times should be in US/East local time
class_start = '2021-04-20'
class_end   = '2021-04-22'

#input your student device name
device_name = 'Guthrum'

#status
print('Global Variables Set')

Global Variables Set

# configuration setup for the notebook libraries and Methods

In [0]:
#import libraries, functions, methods, configurations
#import studentConfig
#status
print('Config Successful')

Config Successful

# read your data into a pandas dataframe

In [0]:
%fs cp dbfs:/FileStore/tables/appIOTcourse_4_allStudent_data_csv.csv file:/tmp/large_file.csv

res1: Boolean = true

In [0]:
print("offline notebook use. Reading data from local .csv file....")

filePath = "/dbfs/FileStore/tables/appIOTcourse_4_allStudent_data_csv.csv"

all_classData = pd.read_csv('file:/tmp/large_file.csv', low_memory=False)

#inspect the schema
all_classData.info()

offline notebook use. Reading data from local .csv file....
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200085 entries, 0 to 200084
Data columns (total 42 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 id 200085 non-null int64 
 1 dtg 200085 non-null object 
 2 name 200085 non-null object 
 3 uuid 200085 non-null object 
 4 source 200085 non-null object 
 5 lat 200085 non-null float64
 6 lon 200085 non-null float64
 7 unix_timestamp 200085 non-null int64 
 8 utc_ts 200085 non-null object 
 9 localized_ts 200085 non-null object 
 10 tzID 200085 non-null object 
 11 platform_country 0 non-null float64
 12 platform_bandwidth 0 non-null float64
 13 accuracy 200085 non-null float64
 14 gps_speed 200085 non-null float64
 15 ipv_4 0 non-null float64
 16 carrier 0 non-null float64
 17 user_agent 0 non-null float64
 18 make 0 non-null float64
 19 model 0 non-null float64
 20 device 0 non-null float64
 21 platform_os 0 non-null float64
 22 app_id 0 non-null float64
 23 publisher_id 0 non-null float64
 24 place_id 0 non-null float64
 25 place_name 0 non-null float64
 26 category 0 non-null float64
 27 gps_src 0 non-null float64
 28 app 0 non-null float64
 29 datasrc_id 0 non-null float64
 30 publisher 0 non-null float64
 31 keywords 0 non-null float64
 32 altitude 200085 non-null float64
 33 speed 200085 non-null float64
 34 latLon_string 21863 non-null object 
 35 valid 21863 non-null object 
 36 type 178222 non-null object 
 37 ssid 45061 non-null object 
 38 mac 178222 non-null object 
 39 rssi 178222 non-null float64
 40 channel 178222 non-null float64
 41 authMode 178134 non-null object 
dtypes: float64(27), int64(2), object(13)
memory usage: 64.1+ MB

## format for time

In [0]:
#format date columns
all_classData['utc_ts'] = all_classData['utc_ts'].astype(str)
all_classData['utc_ts'] =  pd.to_datetime(all_classData['utc_ts'].str.strip(), errors='coerce', utc = True)
#add time zone id
all_classData['tzID'] = 'US/Eastern'
all_classData['localized_ts'] = all_classData['utc_ts'].dt.tz_convert('US/Eastern')
all_classData['dtg'] = all_classData['localized_ts'].dt.date
all_classData['dtg'] = pd.to_datetime(all_classData['dtg'])
all_classData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200085 entries, 0 to 200084
Data columns (total 42 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 id 200085 non-null int64 
 1 dtg 200085 non-null datetime64[ns] 
 2 name 200085 non-null object 
 3 uuid 200085 non-null object 
 4 source 200085 non-null object 
 5 lat 200085 non-null float64 
 6 lon 200085 non-null float64 
 7 unix_timestamp 200085 non-null int64 
 8 utc_ts 200085 non-null datetime64[ns, UTC] 
 9 localized_ts 200085 non-null datetime64[ns, US/Eastern]
 10 tzID 200085 non-null object 
 11 platform_country 0 non-null float64 
 12 platform_bandwidth 0 non-null float64 
 13 accuracy 200085 non-null float64 
 14 gps_speed 200085 non-null float64 
 15 ipv_4 0 non-null float64 
 16 carrier 0 non-null float64 
 17 user_agent 0 non-null float64 
 18 make 0 non-null float64 
 19 model 0 non-null float64 
 20 device 0 non-null float64 
 21 platform_os 0 non-null float64 
 22 app_id 0 non-null float64 
 23 publisher_id 0 non-null float64 
 24 place_id 0 non-null float64 
 25 place_name 0 non-null float64 
 26 category 0 non-null float64 
 27 gps_src 0 non-null float64 
 28 app 0 non-null float64 
 29 datasrc_id 0 non-null float64 
 30 publisher 0 non-null float64 
 31 keywords 0 non-null float64 
 32 altitude 200085 non-null float64 
 33 speed 200085 non-null float64 
 34 latLon_string 21863 non-null object 
 35 valid 21863 non-null object 
 36 type 178222 non-null object 
 37 ssid 45061 non-null object 
 38 mac 178222 non-null object 
 39 rssi 178222 non-null float64 
 40 channel 178222 non-null float64 
 41 authMode 178134 non-null object 
dtypes: datetime64[ns, US/Eastern](1), datetime64[ns, UTC](1), datetime64[ns](1), float64(27), int64(2), object(10)
memory usage: 64.1+ MB

In [0]:
import keplergl
import tempfile
 
orig_repr_html_ = keplergl.KeplerGl._repr_html_
def html_patch(self):
  """This is the patch to make it work well in Databricks"""
  # temp file purposely doesn't get cleaned up, but should if that's desired
  (_, tmp) = tempfile.mkstemp()
  frame_bytes = orig_repr_html_(self)
  frame_str = str(frame_bytes, encoding='utf-8')
  # This additional script for fixing the height is necessary because kepler doesn't embed well, and
  # mutates containing document directly. The height parameter to keplergl.KeplerGl will be the initial
  # height of the result iframe.
  return f"""{frame_str}<script>
    var targetHeight = "{self.height or 600}px";
    var interval = window.setInterval(function() {{
      if (document.body && document.body.style && document.body.style.height !== targetHeight) {{
        document.body.style.height = targetHeight;
      }}
    }}, 250);</script>""";
setattr(keplergl.KeplerGl, '_repr_html_', html_patch)

# inspect your dataframe with Kepler.go

In [0]:
config = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "ud0x28",
          "type": "hexagonId",
          "config": {
            "dataId": "data",
            "label": "h3",
            "color": [
              18,
              147,
              154
            ],
            "columns": {
              "hex_id": "h3"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.8,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "coverage": 1,
              "enable3d": False,
              "sizeRange": [
                0,
                500
              ],
              "coverageRange": [
                0,
                1
              ],
              "elevationScale": 5
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "count",
              "type": "integer"
            },
            "colorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear",
            "coverageField": None,
            "coverageScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "data": [
              {
                "name": "h3",
                "format": None
              },
              {
                "name": "count",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 44.70626352772753,
      "longitude": -101.49221877172097,
      "pitch": 0,
      "zoom": 2.1845063353220064,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

In [0]:
sparkDF=spark.createDataFrame(all_classData) 
sparkDF.createOrReplaceTempView('app_data')
df = spark.sql("select * from app_data")
points_sample = df.sample(False, 0.1, 42)
pdf = points_sample.toPandas()
#pdf = df.toPandas()
map_1 = keplergl.KeplerGl(height=600)
map_1.add_data(data = pdf, name = "app_data")

#displayHTML(map_1._repr_html_())

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter

In [0]:
# format dataframe for plotting
plotAll = all_classData.copy(deep=False)

# plot the data
map_1 = KeplerGl(height=525)
map_1.add_data(data=plotAll, name='all_data')
#map_1.save_to_html(file_name='first_map.html')
#html_string = map_1._repr_html_()
#print(first_map.html)
#h = f"<iframe srcdoc='{html_string}' width=100 height=400 ></iframe>"
#displayHTML(h)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter

# Query for your device mobile ID by the device's name

In [0]:
# lookup the mid based upon the name of the student's device
#check if online or offline mode
if connection == 1:
    #online
    mid = get_mid(device_name,course)
    print("Student mid (online):")
    print(mid)
elif connection == 0:
    #offline
    mask = offline_lookup['name'] == device_name
    row = offline_lookup.loc[mask]
    mid = row.iloc[0]['udid']
    print("Student mid (offline):")
    print(mid)
else:
    #user entry error
    print("connection selection is invalid")

Student mid (offline):
22c07d36-5d29-4064-a3d7-f9019cd27cea

# Query for your data by the mobile ID (UUID)

In [0]:
#display student device data by the mid associated with the device global varible
# The results should be the same as the previous command
mask = all_classData['uuid'] == mid
yourData = all_classData.loc[mask]

print(f"Your devices mid is: {mid}")
yourData.head(3)

Your devices mid is: 22c07d36-5d29-4064-a3d7-f9019cd27cea
Out[15]:

,id,dtg,name,uuid,source,lat,lon,unix_timestamp,utc_ts,localized_ts,tzID,platform_country,platform_bandwidth,accuracy,gps_speed,ipv_4,carrier,user_agent,make,model,device,platform_os,app_id,publisher_id,place_id,place_name,category,gps_src,app,datasrc_id,publisher,keywords,altitude,speed,latLon_string,valid,type,ssid,mac,rssi,channel,authMode
0,1778,2021-04-20,Guthrum,22c07d36-5d29-4064-a3d7-f9019cd27cea,app,38.916472,-77.226884,1618891247,2021-04-20 04:00:47+00:00,2021-04-20 00:00:47-04:00,US/Eastern,NaN,NaN,13.047,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.9,0.4,"38.916472, -77.226885",True,NaN,NaN,NaN,NaN,NaN,NaN
1,1779,2021-04-20,Guthrum,22c07d36-5d29-4064-a3d7-f9019cd27cea,app,38.916475,-77.226873,1618891307,2021-04-20 04:01:47+00:00,2021-04-20 00:01:47-04:00,US/Eastern,NaN,NaN,12.833,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.9,0.0,"38.916475, -77.226873",True,NaN,NaN,NaN,NaN,NaN,NaN
2,1780,2021-04-20,Guthrum,22c07d36-5d29-4064-a3d7-f9019cd27cea,app,38.916474,-77.226872,1618891367,2021-04-20 04:02:47+00:00,2021-04-20 00:02:47-04:00,US/Eastern,NaN,NaN,12.836,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.8,0.1,"38.916474, -77.226872",True,NaN,NaN,NaN,NaN,NaN,NaN


# Let's look at the IoT details captured for you device

In [0]:
#display student device IoT by the device_name global varible
mask = yourData['source'] == 'iot'
yourIOT = yourData.loc[mask]
yourIOT = yourIOT.reset_index()
print(f"IOT Collections on device: {device_name}") 
yourIOT.head(3)

IOT Collections on device: Guthrum
Out[16]:

,index,id,dtg,name,uuid,source,lat,lon,unix_timestamp,utc_ts,localized_ts,tzID,platform_country,platform_bandwidth,accuracy,gps_speed,ipv_4,carrier,user_agent,make,model,device,platform_os,app_id,publisher_id,place_id,place_name,category,gps_src,app,datasrc_id,publisher,keywords,altitude,speed,latLon_string,valid,type,ssid,mac,rssi,channel,authMode
0,21863,23641,2021-04-20,Guthrum,22c07d36-5d29-4064-a3d7-f9019cd27cea,iot,38.916477,-77.226865,1618932008,2021-04-20 15:20:08+00:00,2021-04-20 11:20:08-04:00,US/Eastern,NaN,NaN,117.920006,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.818359,0.0,NaN,NaN,wifi,coxwifi,78:45:61:ab:4d:16,-41.0,11.0,[ess]
1,21864,23642,2021-04-20,Guthrum,22c07d36-5d29-4064-a3d7-f9019cd27cea,iot,38.916477,-77.226865,1618932008,2021-04-20 15:20:08+00:00,2021-04-20 11:20:08-04:00,US/Eastern,NaN,NaN,117.920006,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.818359,0.0,NaN,NaN,wifi,ssid6-2.4,78:45:61:ab:4d:13,-41.0,11.0,[ess]
2,21865,23643,2021-04-20,Guthrum,22c07d36-5d29-4064-a3d7-f9019cd27cea,iot,38.916477,-77.226865,1618932008,2021-04-20 15:20:08+00:00,2021-04-20 11:20:08-04:00,US/Eastern,NaN,NaN,117.920006,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.818359,0.0,NaN,NaN,wifi,ssid4-2.4,78:45:61:ab:4d:12,-41.0,11.0,[ess]


# Let's Look at the APP data

In [0]:
#display student device IoT by the device_name global varible
mask = yourData['source'] == 'app'
yourAPP = yourData.loc[mask]
yourAPP = yourAPP.reset_index()
yourAPP = yourAPP.iloc[:,0:13]
print(f"First party APP collections on device: {device_name}") 
yourAPP.head(3)

First party APP collections on device: Guthrum
Out[17]:

,index,id,dtg,name,uuid,source,lat,lon,unix_timestamp,utc_ts,localized_ts,tzID,platform_country
0,0,1778,2021-04-20,Guthrum,22c07d36-5d29-4064-a3d7-f9019cd27cea,app,38.916472,-77.226884,1618891247,2021-04-20 04:00:47+00:00,2021-04-20 00:00:47-04:00,US/Eastern,NaN
1,1,1779,2021-04-20,Guthrum,22c07d36-5d29-4064-a3d7-f9019cd27cea,app,38.916475,-77.226873,1618891307,2021-04-20 04:01:47+00:00,2021-04-20 00:01:47-04:00,US/Eastern,NaN
2,2,1780,2021-04-20,Guthrum,22c07d36-5d29-4064-a3d7-f9019cd27cea,app,38.916474,-77.226872,1618891367,2021-04-20 04:02:47+00:00,2021-04-20 00:02:47-04:00,US/Eastern,NaN


## Plot your app data

In [0]:
# plot the data
map_2 = KeplerGl(height=525)
map_2.add_data(data=yourAPP, name='yourApp_data')
map_2.config = config_yourData
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Out[18]: KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'o12ushs', 'type': …

# filter your data by velocity

In [0]:
# get dataframe to work on
df = yourAPP.copy(deep=True)

#add a unique ID to dataframe for testing or joining with other dataframes
df.insert(0, 'num', range(0, 0 + len(df)))

#work with time to get the tdoa value in sec and convert it to hours
df['time_delta'] = df.sort_values(['name','utc_ts']).groupby('name')['utc_ts'].diff()
df['time_delta_seconds'] = (df['time_delta'].dt.total_seconds())
df['time_delta_seconds'] = df['time_delta_seconds'].fillna(0)
df['time_delta_hours'] = df['time_delta_seconds'] / 3600
# make the first point (start point of the journey) 0 since it will be a nan
df['time_delta_hours'] = df['time_delta_hours'].fillna(0)
#clean up NaT because kepler does not like NaT when you plot
df.drop('time_delta', axis=1, inplace=True)

# vectorized haversine function
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes (float)

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

#shift df and add a coloumn for the original value of lat and lon
df['orig_lat'] = df.groupby(['name'])['lat'].transform(lambda x:x.shift())
df['orig_lon'] = df.groupby(['name'])['lon'].transform(lambda x:x.shift())
df['dist_km'] = haversine(df['lat'], df['lon'], df['orig_lat'], df['orig_lon'])
# make the first point (start point of the journey) 0 since it will be a NAT
df['dist_km'] = df['dist_km'].fillna(0)
df['dist_miles']=df['dist_km']/1.609
#obtain the velocity
df['velocity_kmph'] = (df['dist_km'] / df['time_delta_hours']).fillna(0)
df['velocity_mph'] = (df['dist_miles'] / df['time_delta_hours']).fillna(0)
#clean temp columns for lat / lon orininals
#df.drop('orig_lat', axis=1, inplace=True)
#df.drop('orig_lon', axis=1, inplace=True)

filter = df[(df.velocity_mph <11)]                             #<------------------------Velocity limit goes here (11)
map_1 = KeplerGl(height=525)
map_1.add_data(data=filter, name='all_data')
map_1.config = config1
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Out[19]: KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'euwj4t8', 'type': …

In [0]:
#filter out your velocity over a certain velocity range
filter = df[(df.velocity_mph >35) & (df.velocity_mph <65)]

# plot the data
map_2 = KeplerGl(height=525)
map_2.add_data(data=filter, name='filter_mph')
map_2.config = config_yourData
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Out[20]: KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'o12ushs', 'type': …

# Inspect your data by source type

In [0]:
# plot the data
map_2 = KeplerGl(height=525)
map_2.add_data(data=yourAPP, name='app_data')
map_2.add_data(data=yourIOT, name='IOT_data')
map_2.config = config_yourData
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Out[21]: KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'o12ushs', 'type': …

# Section 2 | Data Reduction and Visualization Based on DataTime

In [0]:
#Set DateTime Range to Align with Exercise or Short Time Window of Interest
time1 = '2021-04-20 08:05:49+00:00'
time2 = '2021-04-20 08:06:51+00:00'

#Get Dataframoe to work on and set ts column to utc time
plotWindow = yourData.copy(deep=True)

#Filter the DateTime Range
mask = (plotWindow['localized_ts'] > time1) & (plotWindow['localized_ts'] < time2) 
#Run query
filtered_data = plotWindow.loc[mask]
filtered_data.head(10)

Out[22]:

,id,dtg,name,uuid,source,lat,lon,unix_timestamp,utc_ts,localized_ts,tzID,platform_country,platform_bandwidth,accuracy,gps_speed,ipv_4,carrier,user_agent,make,model,device,platform_os,app_id,publisher_id,place_id,place_name,category,gps_src,app,datasrc_id,publisher,keywords,altitude,speed,latLon_string,valid,type,ssid,mac,rssi,channel,authMode
228,2006,2021-04-20,Guthrum,22c07d36-5d29-4064-a3d7-f9019cd27cea,app,38.916472,-77.226873,1618905976,2021-04-20 08:06:16+00:00,2021-04-20 04:06:16-04:00,US/Eastern,NaN,NaN,12.833,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.8,0.1,"38.916472, -77.226873",True,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# plot the data
map_2 = KeplerGl(height=525)
map_2.add_data(data=filtered_data, name='time_filtered_data')
map_2.config = config2
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Out[23]: KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'mdhffdp', 'type': …

# Section 3 | Multi-Device Dataset Import, and Visualize

In [0]:
#Get Dataframoe to work on and obtain all the student name and the number of events observed by each one
allData = all_classData.copy(deep=True)
allData = allData.groupby(['name','uuid']).size().reset_index().rename(columns={0:'count'})
#display the results
allData

Out[24]:

,name,uuid,count
0,Aelfric,801b9140-6179-4eae-b6db-fcbf5788ce0c,37877
1,Brida,bea8836d-3008-44a3-8c3c-53555f0a2dee,23744
2,Guthrum,22c07d36-5d29-4064-a3d7-f9019cd27cea,29529
3,Odda,66946627-b308-4b54-a147-5f4983197b73,17219
4,Ragnar,32bdb1af-fd87-4096-a683-63cd16a264d6,28598
5,Ubba,a266ab79-daf2-4375-a94e-5522a980e934,37561
6,Uhtred,e30234a8-a2f8-4bdd-b82e-3ac919eeec7a,25557


In [0]:
udid1 = 'bea8836d-3008-44a3-8c3c-53555f0a2dee'
udid2 = '32bdb1af-fd87-4096-a683-63cd16a264d6'

#make a local copy of the date to use for plotting
allData1 = all_classData.copy(deep=True)

#select the udid's we want to look at
allData1 = allData1[allData1['uuid'].isin([udid1,udid2])]
allData1['name'].unique()

Out[25]: array(['Ragnar', 'Brida'], dtype=object)

In [0]:
# plot the data
map_3 = KeplerGl(height=525)
map_3.add_data(data=allData1, name='two_devices')
map_3.config = config3
map_3

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Out[26]: KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '0ccvbn', 'type': '…

# Section 4 | Uber H3 HexBins - Associated Locations by Visit Geo [Where Who Where]

#### Geospatial indexing is essential for aggregating and querying data at scale. This type of data is generally abundant, difficult to index or search, and can be structurally complex. Polygons that discriminate particular areas can be very complex to handle. Think of a city’s boundaries. How many points do you need to efficiently determine if a given vehicle has entered a town or even a gas station? The more points you need, the more calculations you will require from your CPU. Overburdening your hardware translates to slower response times and higher resource usage.
#### An efficient geospatial indexing system helps you overcome these hurdles. In the case of H3, the solution takes the form of a hashing scheme.

#### The H3 algorithm partitions the Earth’s surface into a network of hexagons. You can select the amount of detail each hexagon contains by choosing among the available sixteen levels.

In [0]:
# display the resolution available for h3 indexing with hexagons
hexagon_table = h3_indexTable()
hexagon_table

Out[27]:

,edge_length_km,perimeter_km,area_sqkm,edge_length_m,perimeter_m,area_sqm
0,1107.713,6646.276,4250546.848,1107712.591,6646275.546,4.250547e+12
1,418.676,2512.056,607220.978,418676.005,2512056.033,6.072210e+11
2,158.245,949.468,86745.854,158244.656,949467.935,8.674585e+10
3,59.811,358.865,12392.265,59810.858,358865.148,1.239226e+10
4,22.606,135.638,1770.324,22606.379,135638.276,1.770324e+09
5,8.544,51.266,252.903,8544.408,51266.450,2.529034e+08
6,3.229,19.377,36.129,3229.483,19376.897,3.612905e+07
7,1.221,7.324,5.161,1220.630,7323.779,5.161293e+06
8,0.461,2.768,0.737,461.355,2768.128,7.373276e+05
9,0.174,1.046,0.105,174.376,1046.254,1.053325e+05


### To put these areas into context see the below descriptions:
#### *Resolution 7: Roughly city-sized
#### *Resolution 8: Roughly neighborhood-sized
#### *Resolution 9: Roughly 2-4 city blocks in size (different cities have different block sizes)
#### *Resolution 10: Less than a city block in size
#### *Resolution 15: Coffee table-sized

In [0]:
def colocate_modified (df,res):
    #Get Dataframoe to work on and set ts column to utc time
    data_h3 = df.copy(deep=True)
    
    def lat_lng_to_h3(row):
        return h3.geo_to_h3(row['lat'], row['lon'], res)
      
    data_h3.dropna(subset=['lat','lon','localized_ts'], inplace=True)
    data_h3['localized_ts'] = data_h3['localized_ts'].dt.strftime("%X")

    # Create a new Hex ID column describing what H3 hexagon bin each point falls within
    data_h3['hexID'] = data_h3.apply(lat_lng_to_h3 ,axis=1)
    return(data_h3)

### Let's identify co-located passenger in a H3 hexagon at res = 9

In [0]:
# Keep the resolution at 9; the higher the number, the more granular the hexbins become
resolution = 9

#perform the colocation analysis by h3 index binning
hex_colo_modified = colocate_modified(all_classData,resolution)
hex_colo_modified['colocated_passengers'] = (hex_colo_modified.groupby(["hexID","localized_ts"])["name"].transform(lambda x: ', '.join(x.unique())))
hex_colo_modified = hex_colo_modified[hex_colo_modified['colocated_passengers'].str.contains(",")]
hex_colo_minimalfields = hex_colo_modified[['colocated_passengers','hexID','lat','lon','localized_ts','speed']].drop_duplicates().reset_index()
hex_colo_minimalfields.count()

Out[51]: index 8858
colocated_passengers 8858
hexID 8858
lat 8858
lon 8858
localized_ts 8858
speed 8858
dtype: int64

### Now let's identify co-located passenger in a H3 hexagon at res = 15. As you can see, the more the resolution the better the accuracy.

In [0]:
# Keep the resolution at 15; the higher the number, the more granular the hexbins become
resolution = 15

#perform the colocation analysis by h3 index binning
hex_colo_modified_15 = colocate_modified(all_classData,resolution)
hex_colo_modified_15['colocated_passengers'] = (hex_colo_modified_15.groupby(["hexID","localized_ts"])["name"].transform(lambda x: ', '.join(x.unique())))
hex_colo_modified_15 = hex_colo_modified_15[hex_colo_modified_15['colocated_passengers'].str.contains(",")]
hex_colo_modified_15 = hex_colo_modified_15[['colocated_passengers','hexID','lat','lon','localized_ts','speed']].drop_duplicates().reset_index()
hex_colo_modified_15.count()

Out[49]: index 377
colocated_passengers 377
hexID 377
lat 377
lon 377
localized_ts 377
speed 377
dtype: int64

In [0]:
hex_colo_modified_15.head(50)

Out[50]:

,index,colocated_passengers,hexID,lat,lon,localized_ts,speed
0,26,"Guthrum, Uhtred",8f2aa84d3a5cce5,38.916474,-77.226871,00:28:27,0.1
1,41,"Guthrum, Uhtred",8f2aa84d3a5cce0,38.916476,-77.226876,00:44:34,0.0
2,46,"Guthrum, Odda",8f2aa84d3a5cce0,38.916473,-77.226879,00:50:00,0.1
3,412,"Guthrum, Odda",8f2aa84d3a5cce0,38.916477,-77.226878,07:24:12,0.1
4,420,"Guthrum, Uhtred",8f2aa84d3a5cce0,38.916475,-77.226878,07:32:49,0.1
5,1112,"Guthrum, Ubba",8f2aa84d3a5cce4,38.916482,-77.226871,11:22:17,0.2
6,2699,"Guthrum, Ragnar",8f2aa84d3a5cc19,38.916488,-77.226880,15:53:57,0.1
7,2726,"Guthrum, Ragnar",8f2aa84d3a5cc0a,38.916493,-77.226874,16:00:35,0.2
8,2728,"Guthrum, Uhtred",8f2aa84d3a5cc19,38.916492,-77.226877,16:01:18,0.0
9,2786,"Guthrum, Uhtred",8f2aa84d3a5cc19,38.916489,-77.226885,16:15:28,0.1


In [0]:
# plot the data
hexagons = pd.DataFrame(hex_colo_minimalfields.hexID)
colo_devices = pd.DataFrame(hex_colo_minimalfields[['hexID','colocated_passengers','lat','lon','speed']])

map_4 = KeplerGl(height=525)
map_4.add_data(data=colo_devices, name='colo_devices')
map_4.add_data(data=hexagons, name='hex_res9')
map_4.config = config1
displayHTML(map_4._repr_html_())

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter

<!doctype html> Kepler.gl

# Section 5 | Graphical Visualizatin Comparing UUIDs and SSIDs [Technological Proximity]

In [0]:
#Need a graph example for people being connected together in time and space